# ARGO_GDAC profile check

This script will read in the synthetic profile

1) read the raw and adjusted o2 data in 'all good' status (PROFILE_DOXY_QC = 'A')

2) interpolate the data onto standard depth levels (WOA2009 standard z level)

3) compare it against the WOA09 data (match up with the nearest 1x1 climatology grid) 

4) calculate RMSE overall for each float sensor

5) save interpolated profile data as netCDF file

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
from scipy import interpolate as itp
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# set up standard depth, year, month
zstd=np.array([0,10,20,30,50,75,100,125,150,200,250,300,400,
               500,600,700,800,900,1000,1100,1200,1300,1400,
               1500,1750,2000])
year=np.arange(2011,2020,1)
month=np.arange(1,13,1)

In [3]:
# generate index for downloaded synthetic profiles
os.system('ls data/*.nc > available_floats.txt')
fn=pd.read_csv('available_floats.txt',header=None,names=['file'])

In [9]:
# open WOA data files
dmon=xr.open_dataset('WOA09/dissolved_oxygen_monthly_1deg.nc',decode_times=False)
dann=xr.open_dataset('WOA09/dissolved_oxygen_annual_1deg.nc',decode_times=False)

In [20]:
# open the profile data file, count the number of profiles and depth levels
N=np.size(fn)
Nprf=np.empty((N,))
Nlev=np.empty((N,))
o2rmse=np.empty((N,))
o2rmse500=np.empty((N,))
o2rmse1000=np.empty((N,))
o2rmse_adjusted=np.empty((N,))
o2rmse500_adjusted=np.empty((N,))
o2rmse1000_adjusted=np.empty((N,))
file_output=np.empty((N,),np.dtype('U100'))

! mkdir -p compare_woa

# loop over all floats
for n in range(0,N):
    if (np.remainder(n,50)==0):
        print('completed '+str(n)+' floats')
        
    ds=xr.open_dataset(fn['file'][n])
    Ntmp=np.shape(ds['PRES'])
    Nprf[n]=Ntmp[0]
    Nlev[n]=Ntmp[1]
    
    pid=ds['PLATFORM_NUMBER'][0].astype(str)
    o2=ds['DOXY'].astype(float)
    o2a=ds['DOXY_ADJUSTED'].astype(float)
    z=ds['PRES'].astype(float)
    x=ds['LONGITUDE'].astype(float)
    y=ds['LATITUDE'].astype(float)
    date=ds['JULD']
    
    # get QC flags
    date_qc=ds['JULD_QC'].astype(int)
    pos_qc=ds['POSITION_QC'].astype(int)
    o2_qc=ds['PROFILE_DOXY_QC'].astype(str)
    o2qcflg=ds['DOXY_QC'].astype(str)
    o2aqcflg=ds['DOXY_ADJUSTED_QC'].astype(str)
    
    pdate=pd.to_datetime(date)
    year=pdate.year
    mon=pdate.month
#    goodprf=(pos_qc==1) & (date_qc==1) & ((o2_qc=='A') | (o2_qc=='B'))
    goodprf=(pos_qc==1) & (date_qc==1)
    
    # prepare o2woa data array for this particular float
    o2woa=xr.DataArray( np.empty((int(Nprf[n]),26)) ,  
                       [("profile",range(0,int(Nprf[n]))) , 
                        ("depth",zstd) 
                       ],name='O2_WOA')
    o2flt=xr.DataArray( np.empty((int(Nprf[n]),26)) ,  
                       [("profile",range(0,int(Nprf[n]))) , 
                        ("depth",zstd) 
                       ],name='O2_FLT')
    o2flt_adjusted=xr.DataArray( np.empty((int(Nprf[n]),26)) ,  
                       [("profile",range(0,int(Nprf[n]))) , 
                        ("depth",zstd) 
                       ],name='O2_FLT_ADJUSTED')

    # fill in the WOA09 data
    for m in range(0,int(Nprf[n])):
        
        if ((goodprf[m]==True) & ((x[m]+y[m])<450)):
            # co-locate WOA profile
            # print('good profile detected')
            o2woa_ann=dann.o_an.sel(lon=x[m],lat=y[m],method='nearest')*43.570
            tmp=dmon.o_an.sel(lon=x[m],lat=y[m],method='nearest')*43.570
            o2woa_mon=tmp[int(mon[m]-1),:]
            o2woa[m,:26]=o2woa_ann.data[0,0:26]
            o2woa[m,:24]=o2woa_mon.data
            
            # get the profile data
            oflg=o2qcflg[m,:]
            ogood=((oflg=='1')|(oflg=='2')|(oflg=='5')|(oflg=='8'))
            o20=o2[m,ogood]
            z0=z[m,ogood]
            
            # linearly interpolate
            if (np.size(o20)>1):
                fo2=itp.interp1d(z0,o20,'linear',bounds_error=False,fill_value=np.nan)
                o2flt[m,:]=fo2(zstd)
            else:
                o2flt[m,:]=np.nan
                
            # get the adjusted profile data
            oflg=o2aqcflg[m,:]
            ogood=((oflg=='1')|(oflg=='2')|(oflg=='5')|(oflg=='8'))
            o20=o2a[m,ogood]
            z0=z[m,ogood]
             
            # linearly interpolate
            if (np.size(o20)>1):
                fo2=itp.interp1d(z0,o20,'linear',bounds_error=False,fill_value=np.nan)
                o2flt_adjusted[m,:]=fo2(zstd)
            else:
                o2flt_adjusted[m,:]=np.nan
             
        else:
            o2woa[m,:]=np.nan
            o2flt[m,:]=np.nan
            o2flt_adjusted[m,:]=np.nan
            
    o2dif=o2flt-o2woa
    o2dif_adjusted=o2flt_adjusted-o2woa
    
    o2rmse[n]=np.sqrt((o2dif**2).mean(dim='profile').mean(dim='depth'))
    o2rmse500[n]=np.sqrt((o2dif[:,14:]**2).mean(dim='profile').mean(dim='depth'))
    o2rmse1000[n]=np.sqrt((o2dif[:,19:]**2).mean(dim='profile').mean(dim='depth'))
   
    o2rmse_adjusted[n]=np.sqrt((o2dif_adjusted**2).mean(dim='profile').mean(dim='depth'))
    o2rmse500_adjusted[n]=np.sqrt((o2dif_adjusted[:,14:]**2).mean(dim='profile').mean(dim='depth'))
    o2rmse1000_adjusted[n]=np.sqrt((o2dif_adjusted[:,19:]**2).mean(dim='profile').mean(dim='depth'))

    # optional: merge data arrays into a new dataset and save it in netcdf format 
    if (sum(goodprf)>=1):
        ds_out=xr.merge([o2woa,o2flt,o2flt_adjusted,x,y,date])
        wn='compare_woa/'+str(np.char.strip(pid.data))+'_26lev.nc'
        ds_out.to_netcdf(wn,mode='w')
        ds_out
        file_output[n]=wn

completed 0 floats
completed 50 floats
completed 100 floats
completed 150 floats
completed 200 floats
completed 250 floats
completed 300 floats
completed 350 floats
completed 400 floats
completed 450 floats
completed 500 floats
completed 550 floats
completed 600 floats
completed 650 floats
completed 700 floats
completed 750 floats
completed 800 floats
completed 850 floats
completed 900 floats
completed 950 floats
completed 1000 floats
completed 1050 floats
completed 1100 floats
completed 1150 floats
completed 1200 floats
completed 1250 floats
completed 1300 floats
completed 1350 floats
completed 1400 floats
completed 1450 floats
completed 1500 floats


In [23]:
a=np.array([Nprf, o2rmse, o2rmse500, o2rmse1000, o2rmse_adjusted, 
            o2rmse500_adjusted, o2rmse1000_adjusted, file_output])
float_stats=pd.DataFrame(data=a.T,
                        columns=['Num Prof','RMSE(0-2000m)','RMSE(500-2000m)','RMSE(1000-2000m)'
                                 ,'RMSE_ADJ(0-2000m)','RMSE_ADJ(500-2000m)','RMSE_ADJ(1000-2000m)','File'])
float_stats.to_csv('float_summary_statsitics.csv')